In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import missingno as msno
import pandas_profiling
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
#解决中文显示问题
plt.rcParams["font.sans-serif"] = [u"SimHei"]
plt.rcParams["axes.unicode_minus"] = False
#解决图中负号问题
plt.rcParams['axes.unicode_minus']=False

In [ ]:
column_name=['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg','os','battery_capacity','storage','screen_size']

# names = ['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg']
xy_1 = pd.read_table('ZZ0702GuangFa_Samples100000_FeatureData.txt',names=column_name)
xy_1.head()

In [ ]:
xy_1.info()

In [ ]:
plt.style.use('bmh')
plt.rc('font',family = 'SimHei',size = 13)
#cat_list = ['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg']
cat_list=column_name
for n,i in enumerate(cat_list):
    xy_1_cat_num = xy_1[i].value_counts().index.shape[0]
    print('{0}.{1}特征的类型数量是：{2}'.format(n+1,i,xy_1_cat_num))

In [ ]:
# 去除重复行：
xy_1.drop_duplicates(keep='first',inplace=True)
xy_1.reset_index(inplace=True)

In [ ]:
msno.matrix(xy_1,labels = True)
xy_1.shape

In [ ]:
pandas_profiling.ProfileReport(xy_1)

In [ ]:
#根据手机上市时间衍生出，上市到现在的累计月份变量listing_data_months
#核心代码，转换，array转换成str使用[:]，转换成int加减
def month_sept(data):
    lst1=[]
    for n in range(len(data)):
        a=data.loc[n]['listing_date']
        if '2019' in str(a):
            month_num = str(a)[4:6]
            lst1.append(month_num)
        elif '2018' in str(a):
            month_num = 12-int(str(a)[4:6])+4
            lst1.append(month_num)
        elif( '2017' in str(a)) and (str(a)!='2017.0'):
            month_num = 12-int(str(a)[4:6])+4+12
            lst1.append(month_num)
        elif '2016' in str(a)and (str(a)!='2016.0'):
            month_num = 12-int(str(a)[4:6])+4+24
            lst1.append(month_num)
        elif '2015' in str(a)and (str(a)!='2015.0'):
            month_num = 12-int(str(a)[4:6])+4+36
            lst1.append(month_num)
        elif '2014' in str(a)and (str(a)!='2014.0'):
            month_num = 12-int(str(a)[4:6])+4+48
            lst1.append(month_num)
        elif '2013' in str(a) and(str(a)!='2013.0'):
            month_num = 12-int(str(a)[4:6])+4+60
            lst1.append(month_num)
    c= pd.concat([xy_1,pd.DataFrame(lst1)],axis=1)
    c.rename(columns={0:'listing_data_months'},inplace = True)
    return c
xy_2 = month_sept(xy_1)

In [ ]:
xy_3 = pd.read_table('city_ranking.txt',names = ['owner_city','owner_city_level'])
xy_5 = pd.merge(xy_2,xy_3,on='owner_city',how = 'left')

In [ ]:
xy_5.columns

In [ ]:
usefulcolumnlist=['target', 'card_type', 'netlong', 'owner_city', 'arpu',
       'out_prdct_fee', 'prdct_fee', 'point_fee', 'brand',
       'listing_price', 'user_level', 'sex',
       'family_client', 'group_client', 'music', 
       'video_member','finance', 'bank',
       'arpu_avg',
       'out_prdct_fee_avg', 'prdct_fee_avg', 'battery_capacity',
       'storage', 'screen_size', 'listing_data_months', 'owner_city_level']

xy_6 = xy_5[usefulcolumnlist]


In [ ]:
xy_6[['battery_capacity','storage','screen_size']].head()

In [ ]:
xy_6['battery_capacity'].value_counts()

def DealBatteryFormat(RowSeriers):
    import re
    if RowSeriers.battery_capacity!=RowSeriers.battery_capacity:
        return 9999
    else:
        Battery = RowSeriers.battery_capacity
        Batterynum = re.findall('\d+',Battery)[0]
        return int(Batterynum)

xy_6['battery_capacity']=xy_6.apply(DealBatteryFormat,axis=1)

In [ ]:
xy_6['battery_capacity'].value_counts().head()

In [ ]:
def DealstorageFormat(RowSeriers):
    storagestr=str(RowSeriers.storage)
   
    if 'G' in storagestr:
        storagesplit=storagestr.split('/')[0]
        if 'G' in storagesplit:
            
            return storagesplit.split('G')[0]+'GB'
        else:            
            return storagesplit+'GB'
    if 'M' in storagestr:
        storagesplit=storagestr.split('/')[0]
        if 'M' in storagesplit:
            return storagesplit.split('M')[0]+'MB'
        else:
            return storagesplit+'MB'
    else:
        return storagestr    

xy_6['storage']=xy_6.apply(DealstorageFormat,axis=1)

In [ ]:
xy_6['battery_capacity'].value_counts()/len(xy_6['battery_capacity'])

In [ ]:
xy_6['finance'].value_counts()
xy_6['finance'] = xy_6['finance'].fillna(9999)
xy_6['finance'].value_counts()

xy_6['bank'].value_counts()
xy_6['bank'] = xy_6['bank'].fillna(9999)
xy_6['bank'].value_counts()

xy_6['owner_city'].value_counts()
xy_6['owner_city'] = xy_6['owner_city'].fillna(9999)
xy_6['owner_city'].value_counts()

xy_6['card_type'].value_counts()
xy_6['card_type'] = xy_6['card_type'].fillna(9999)
xy_6['card_type'].value_counts()

xy_6['netlong'].value_counts()
xy_6['netlong'] = xy_6['netlong'].fillna(9999)
xy_6['netlong'].value_counts()

# arpu 空值填充为999 ，异常值处理
xy_6['arpu'].value_counts()
xy_6['arpu'].describe()
xy_6['arpu'][xy_6['arpu']>500].value_counts()#u+1.5标准差3000
#xy_6['arpu'][xy_6['arpu']>500] = 41.82#中位数替换异常值
xy_6['arpu'] = xy_6['arpu'].fillna(9999)
xy_6['arpu'].value_counts()
# arpu_avg
xy_6['arpu_avg'].value_counts()#空值填充为999 ，异常值处理
xy_6['arpu_avg'].describe()
xy_6['arpu_avg'][xy_6['arpu_avg']>3000].value_counts()#u+1.5标准差3000
#xy_6['arpu_avg'][xy_6['arpu_avg']>3000] = 43.54#中位数替换异常值
xy_6['arpu_avg'] = xy_6['arpu_avg'].fillna(9999)
xy_6['arpu_avg'].value_counts()

# out_prdct_fee 空值填充为999，异常值处理,Nan没超过30%，按时0值超过了50%，直接构造成（空，0，大于0三类）
xy_6['out_prdct_fee'].value_counts()
xy_6['out_prdct_fee'].describe()
xy_6['out_prdct_fee'][xy_6['out_prdct_fee']>0] = 1
xy_6['out_prdct_fee'] = xy_6['out_prdct_fee'].fillna(9999)
xy_6['out_prdct_fee'].value_counts()
# out_prdct_fee_avg Nan超过30%，直接构造成（空，0，大于0三类）
xy_6['out_prdct_fee_avg'].value_counts()
xy_6['out_prdct_fee_avg'].describe()
xy_6['out_prdct_fee_avg'][xy_6['out_prdct_fee_avg']>0] = 1
xy_6['out_prdct_fee_avg'] = xy_6['out_prdct_fee_avg'].fillna(9999)
xy_6['out_prdct_fee_avg'].value_counts()

# prdct_fee 空值填充为999，异常值处理,Nan没超过30%，加上0值未超过50%，加一层空值999
xy_6['prdct_fee'].value_counts()
# xy_6 = xy_6.drop(xy_6['prdct_fee'][xy_6['prdct_fee']==0])#删除0值后看剩下的上四分位点加上1.5倍（999-18）=2500
# xy_6['prdct_fee'].describe()
xy_6['prdct_fee'][xy_6['prdct_fee']>500].value_counts()#中位数替换异常值
#xy_6['prdct_fee'][xy_6['prdct_fee']>500] = 23
xy_6['prdct_fee'] = xy_6['prdct_fee'].fillna(9999)
xy_6['prdct_fee'].value_counts()
# prdct_fee_avg空值填充为999，异常值处理,Nan没超过30%，加上0值（7232）未超过50%，加一层空值999
xy_6['prdct_fee_avg'].value_counts()
# xy_6 = xy_6.drop(xy_6['prdct_fee_avg'][xy_6['prdct_fee_avg']==0])#删除0值后看剩下的上四分位点加上1.5倍（999-18）=2500
xy_6['prdct_fee_avg'].describe()
xy_6['prdct_fee_avg'][xy_6['prdct_fee_avg']>500].value_counts()#中位数替换异常值
#xy_6['prdct_fee_avg'][xy_6['prdct_fee_avg']>500] = 24.6
xy_6['prdct_fee_avg'] = xy_6['prdct_fee_avg'].fillna(9999)
xy_6['prdct_fee_avg'].value_counts()

# point_fee 空值填充为999，异常值处理,Nan没超过30%，加上0值（27824）超过50%，构造（空，0，大于0三类）
xy_6['point_fee'].value_counts()
xy_6['point_fee'][xy_6['point_fee']>0] = 1
xy_6['point_fee'] = xy_6['point_fee'].fillna(9999)
xy_6['point_fee'].value_counts()

# sex 构造3类
xy_6['sex'].value_counts()
xy_6['sex'] = xy_6['sex'].fillna(9999)
xy_6['sex'].value_counts()

# family_client Nan超过30%，构造（空，1两类）
xy_6['family_client'].value_counts()
xy_6['family_client'] = xy_6['family_client'].fillna(9999)
xy_6['family_client'].value_counts()

# group_client  Nan超过30%，构造（空，1两类）
xy_6['group_client'].value_counts()
xy_6['group_client'] = xy_6['group_client'].fillna(9999)
xy_6['group_client'].value_counts()

# music  Nan超过30%，构造（空，1两类）
xy_6['music'].value_counts()
xy_6['music'] = xy_6['music'].fillna(9999)
xy_6['music'].value_counts()

# video_member Nan超过30%，构造（空，1两类）
xy_6['video_member'].value_counts()
xy_6['video_member'][xy_6['video_member'].notnull()] = 1
xy_6['video_member'] = xy_6['video_member'].fillna(9999)
xy_6['video_member'].value_counts()


# listing_data_months 空值填充为999，异常值处理,Nan没超过30%，加上0值未超过50%，加一层空值999
xy_6['listing_data_months'].value_counts()
xy_6['listing_data_months'] = xy_6['listing_data_months'].fillna(9999)
xy_6['listing_data_months'].value_counts()

# user_level  空值填充为999，异常值处理,Nan没超过30%，加上0值未超过50%，加一层空值999
xy_6['user_level'].value_counts()
xy_6['user_level'] = xy_6['user_level'].fillna(9999)
xy_6['user_level'].value_counts()

# listing_price 空值填充为999，异常值处理,Nan没超过30%，加上0值（7232）未超过50%，加一层空值999
xy_6['listing_price'].value_counts()
xy_6['listing_price'] = xy_6['listing_price'].fillna(9999)
xy_6['listing_price'].value_counts()

# brand 空值填充为999，异常值处理,Nan没超过30%，加上0值（7232）未超过50%，加一层空值999
xy_6['brand'].value_counts()
xy_6['brand'] = xy_6['brand'].fillna(9999)
xy_6['brand'].value_counts()

# owner_city_level
xy_6['owner_city_level'].value_counts()
xy_6['owner_city_level'] = xy_6['owner_city_level'].fillna(9999)
xy_6['owner_city_level'].value_counts()
#以上，异常值，空值处理完毕

xy_6['brand'].value_counts()
xy_6['brand'][xy_6['brand']==9999] = '空值'
xy_6['brand'][~xy_6['brand'].isin(['vivo','OPPO','苹果','小米','华为','华为荣耀','三星','金立','酷派','魅族','空值'])] ='其它品牌'
xy_6['brand'].value_counts()

# ['battery_capacity','storage','screen_size']
xy_6['battery_capacity'] = xy_6['battery_capacity'].fillna(9999)
xy_6['battery_capacity'].value_counts()
# xy_6['battery_capacity'][xy_6['battery_capacity']=='nan'] = '空值'
# xy_6['battery_capacity'][~xy_6['battery_capacity'].isin(['3000mah','nan','4000mah','1715mah','3050mah','2000mah','1800mah','2750mah','2910mah','1960mah','2900mah','2400mah','3010mah','3100mah','2200mah','2930mah','2850mah','2600mah','2300mah','4100mah','2550mah','3075mah','2500mah','1900mah','2350mah'])] ='其它电池'

xy_6['storage'] = xy_6['storage'].fillna('空值')
xy_6['storage'][~xy_6['storage'].isin(['nan','16GB','32GB','64GB','8GB','4GB','32MB','128MB','512MB','64MB','128GB','256MB','8MB','4MB','16MB'])] ='其它存储'

xy_6['screen_size'] = xy_6['screen_size'].fillna(99)
#xy_6['screen_size']=xy_6['screen_size'].astype(str)
#xy_6['screen_size'][~xy_6['screen_size'].isin(['5.5','4.7','5','5.2','4','6','5.7','5.8','4.5','5.1','5.9','3.5','空值'])] ='其它屏幕'


# #以上，异常值，空值处理完毕

In [ ]:
msno.matrix(xy_6,labels = True)

In [ ]:
pandas_profiling.ProfileReport(xy_6)

In [ ]:
xy_6.columns

In [ ]:
usefulcolumn2=['target','arpu','battery_capacity','brand','card_type','family_client','finance','group_client','listing_data_months','listing_price','music','netlong','out_prdct_fee','out_prdct_fee_avg','owner_city','owner_city_level','prdct_fee','screen_size','sex','storage','user_level','video_member']

xy_6=xy_6[usefulcolumn2]

xy_6.drop_duplicates(keep='first',inplace=True)
xy_6.reset_index(inplace=True)

In [ ]:
for columns in usefulcolumn2:
    print(columns,len(xy_6[columns].value_counts()))

In [ ]:
#核心代码：遍历版分箱操作
def column_binning(data):
    column_list = ['arpu','battery_capacity','listing_data_months','listing_price','netlong','prdct_fee']
    xy_10 = data
    for column in column_list:
        print(column)
        xy_10.insert(2,column+'_derive_1',xy_10[column])
        xy_7 = xy_10[(xy_10[column+'_derive_1'] == 9999) | (xy_10[column+'_derive_1'] == 0)]
        xy_8 = xy_10[(0<xy_10[column+'_derive_1'])&(xy_10[column+'_derive_1']<9999)]
        xy_8[column+'_derive_1'] = pd.qcut(xy_8[column+'_derive_1'],8)
        xy_9 = pd.concat([xy_8,xy_7],axis = 0)
        xy_10 = xy_9#循环一次的结果作为下一次循环的开始数据，不能直接给data复制，所以需要有xy_10这个中间变量
    return xy_9
xy_9=column_binning(xy_6)

In [ ]:
xy_9.to_excel('20190702广发_信用卡_分箱.xlsx',index = None)

In [ ]:
xy_9.columns

In [ ]:
ChuanQi_basic_df=xy_9
ChuanQi_basic_df=ChuanQi_basic_df[['target', 'prdct_fee_derive_1', 'netlong_derive_1',
       'listing_price_derive_1', 'listing_data_months_derive_1',
       'battery_capacity_derive_1', 'arpu_derive_1', 'arpu',
       'battery_capacity', 'brand', 'card_type', 'family_client', 'finance',
       'group_client', 'listing_data_months', 'listing_price', 'music',
       'netlong', 'out_prdct_fee', 'out_prdct_fee_avg', 'owner_city',
       'owner_city_level', 'prdct_fee', 'screen_size', 'sex', 'storage',
       'user_level', 'video_member']]

ChuanQi_basic_df.reset_index(inplace=True,drop=True)
display(ChuanQi_basic_df.head(),ChuanQi_basic_df.shape)

In [ ]:
def DealBrandOPP0(RowSeries):
    BrandName=RowSeries.brand
    if BrandName in ['OPPO','OPPO ']:
        return 'OPPO'
    else:
        return BrandName
ChuanQi_basic_df.brand=ChuanQi_basic_df.apply(DealBrandOPP0,axis=1)

brand_group=ChuanQi_basic_df.groupby('target')['brand']

In [ ]:
def brand_num(ChuanQi_basic_df):
    title_mapping = {'苹果':1,'华为':2,'小米':3,'华为荣耀':4,'三星':5,'OPPO':6,'vivo':7,'魅族':8,'金立':9,'酷派':10,'空值':11,'其它品牌':12}
    ChuanQi_basic_df['brand'] = ChuanQi_basic_df['brand'].map(title_mapping)
brand_num(ChuanQi_basic_df)
ChuanQi_basic_df.columns

In [ ]:
import numpy as np
#convernetlong_derive_2	listing_price_derive_2
#interval_list=['listing_data_months_derive_1','prdct_fee_avg_derive_1','arpu_avg_derive_1','prdct_fee_derive_1','listing_price_derive_1','arpu_derive_1','netlong_derive_1']
# 传奇游戏特征选择后的连续值：
#interval_list=['arpu_derive_1','listing_price','prdct_fee','arpu_avg','prdct_fee_avg','listing_data_months']

# 网贷游戏特征选择后的连续变量：
interval_list=['prdct_fee_derive_1', 'netlong_derive_1',
       'listing_price_derive_1', 'listing_data_months_derive_1',
       'battery_capacity_derive_1', 'arpu_derive_1',]
# 将区间值列的中间对应值：

intervalcolumn_df={}

def ConverIntervalData():
    for intervalcolumn in interval_list:
        ChuanQi_basic_df[intervalcolumn]=ChuanQi_basic_df[intervalcolumn].astype('str')
        intervalunique=ChuanQi_basic_df[intervalcolumn].unique()
        #print(intervalunique)
        intervalcolumn_df[intervalcolumn]={'intervaluniquelist':[]}
        intervalcolumn_df[intervalcolumn]['intervaluniquelist']=intervalunique
        intervalcolumn_df[intervalcolumn]['intervalValueDict']={}
        for intervalValue in intervalunique:
            intervalLeft=0.0
            intervalRight=0.0
            #print(intervalValue)
            if ', ' in intervalValue:
                intervalValueSplit=intervalValue.split(', ')
                intervalLeft=float(intervalValueSplit[0].split('(')[1])
                intervalRight=float(intervalValueSplit[1].split(']')[0])
                #print(intervalValueSplit,intervalLeft,intervalRight)
                intervalcolumn_df[intervalcolumn]['intervalValueDict'][intervalValue]=(intervalRight+intervalLeft)/2
            else:
                intervalLeft=float(intervalValue)
                intervalRight=float(intervalValue)
                intervalcolumn_df[intervalcolumn]['intervalValueDict'][intervalValue]=(intervalRight+intervalLeft)/2
        
        intervalcolumnAvg=[]
        for intervalValuekey in intervalcolumn_df[intervalcolumn]['intervalValueDict']:
            intervalcolumnAvg.append(intervalcolumn_df[intervalcolumn]['intervalValueDict'][intervalValuekey])
        intervalcolumnAvg=np.array(intervalcolumnAvg)
        intervalcolumnAvgSortIndex=intervalcolumnAvg.argsort()
        maxpointvalue=(intervalcolumnAvg[intervalcolumnAvgSortIndex[-2]]-intervalcolumnAvg[intervalcolumnAvgSortIndex[-3]])/2+intervalcolumnAvg[intervalcolumnAvgSortIndex[-2]]
        intervalcolumn_df[intervalcolumn]['intervalValueDict']['9999']=maxpointvalue
        #print(intervalcolumnAvg,maxpointvalue)      
ConverIntervalData() 

In [ ]:
# 将interval 列的数据进行替换操作：

def ReplaceIntervalColumnValue():
    for intervalcolumn in interval_list:
        ChuanQi_basic_df[intervalcolumn].replace(intervalcolumn_df[intervalcolumn]['intervalValueDict'],inplace=True)
        #print(a.head(10),ChuanQi_basic_df[intervalcolumn].head(10))
ReplaceIntervalColumnValue()

In [ ]:
display(ChuanQi_basic_df.head(),ChuanQi_basic_df.columns)

In [ ]:
ChuanQi_basic_df.to_excel('20190702广发_信用卡_清洗后.xlsx',index=None)

In [ ]:
def ClassifyFeedBack(classifycolumnlist,origindf):
    columnvaluecount_dict={}
    for classifycolumn in classifycolumnlist:
        columngroupvaluecount=origindf.groupby('target')[classifycolumn].value_counts()
        columnvaluecount=origindf[classifycolumn].value_counts()
        if classifycolumn not in columnvaluecount_dict:            
            columnvaluecount_dict[classifycolumn]={}
            columnvaluecount_dict[classifycolumn]['target_group']=columngroupvaluecount
        columnvaluecount_dict[classifycolumn]['columnvaluecount']=columnvaluecount
    return columnvaluecount_dict  

In [ ]:
usefualcolumns=['target', 'prdct_fee_derive_1', 'netlong_derive_1',
       'listing_price_derive_1', 'listing_data_months_derive_1',
       'battery_capacity_derive_1', 'arpu_derive_1', 'arpu',
       'battery_capacity', 'brand', 'card_type', 'family_client', 'finance',
       'group_client', 'listing_data_months', 'listing_price', 'music',
       'netlong', 'out_prdct_fee', 'out_prdct_fee_avg', 'owner_city',
       'owner_city_level', 'prdct_fee', 'screen_size', 'sex', 'storage',
       'user_level', 'video_member']
ChuanQi_basic_df=ChuanQi_basic_df[usefualcolumns]

In [ ]:
for columns in usefualcolumns:
    print(columns,len(ChuanQi_basic_df[columns].value_counts()))

In [ ]:
classifycolumnlist=[ 'prdct_fee_derive_1', 'netlong_derive_1',
       'listing_price_derive_1', 'listing_data_months_derive_1',
       'battery_capacity_derive_1', 'arpu_derive_1',
       'brand', 'card_type', 'family_client', 'finance',
       'group_client', 'listing_data_months', 'music',
       'netlong', 'out_prdct_fee', 'out_prdct_fee_avg', 'owner_city',
       'owner_city_level', 'screen_size', 'sex', 'storage',
       'user_level', 'video_member']
# classifycolumnlist=ChuanQi_basic_df.columns.tolist()
# classifycolumnlist.remove('id')
# classifycolumnlist.remove('target')
columnvaluecount_dict=ClassifyFeedBack(classifycolumnlist,ChuanQi_basic_df)

In [ ]:
# 写个程序检索columnvaluecount_dict 中的不同分类列是否数据一致且要保证相同的特征的不同分区都必须必修要有覆盖，防止分区为空，或者人数为零···

def checkmatchCGVC_dict(columnvaluecount_dict):
    for column in columnvaluecount_dict:
        ColumnAllGroupsIndex=columnvaluecount_dict[column]['columnvaluecount'].index
        TureSampleGroupsIndex=columnvaluecount_dict[column]['target_group'][1].index
        for GroupsIndex in ColumnAllGroupsIndex:
            if GroupsIndex not in TureSampleGroupsIndex: 
                if GroupsIndex not in columnvaluecount_dict[column]['target_group'][1].index:
                    columnvaluecount_dict[column]['target_group'][1,GroupsIndex]=0.00001

    return columnvaluecount_dict

columnvaluecount_dict=checkmatchCGVC_dict(columnvaluecount_dict)

In [ ]:
# 计算正样本的反馈率···
import numpy as np
def CalculateTurnSampleFeedBack(columnvaluecount_dict):
    for column in columnvaluecount_dict:
        columnvaluecount_dict[column]['TurnSampleFeedBack']=columnvaluecount_dict[column]['target_group'][1]/columnvaluecount_dict[column]['columnvaluecount']
        columnvaluecount_dict[column]['TurnSampleLogPercentRadio']=np.log(columnvaluecount_dict[column]['TurnSampleFeedBack']/(1-columnvaluecount_dict[column]['TurnSampleFeedBack']))
    return columnvaluecount_dict

columnvaluecount_dict=CalculateTurnSampleFeedBack(columnvaluecount_dict)
display(columnvaluecount_dict['brand']['TurnSampleLogPercentRadio'].sort_values(ascending=False),columnvaluecount_dict['brand']['TurnSampleLogPercentRadio'][1],)

In [ ]:
# 对不同基础属性的类别值（分区）计算row_percent

def CalculateColumnRowPercent(columnvaluecount_dict):
    for column in columnvaluecount_dict:
        columnvaluecount_dict[column]['ColumnRowPercent']=columnvaluecount_dict[column]['columnvaluecount']/columnvaluecount_dict[column]['columnvaluecount'].sum()
        
    return columnvaluecount_dict

columnvaluecount_dict=CalculateColumnRowPercent(columnvaluecount_dict)

In [ ]:
# 将每个列的反馈率数值及覆盖率值输出出来···
def CreateColumnStatistics(columnvaluecount_dict):
    for column in columnvaluecount_dict:
        columnvaluecount_dict[column]['ColumnStatistics']=pd.DataFrame({'RowPercent':columnvaluecount_dict[column]['TurnSampleFeedBack'],'Percent':columnvaluecount_dict[column]['ColumnRowPercent']})
        
    return columnvaluecount_dict

columnvaluecount_dict=CreateColumnStatistics(columnvaluecount_dict)
len(columnvaluecount_dict)

In [ ]:
# 输出分类型变量的反馈率及覆盖率：
#ClassifyColumnList=['card_type','owner_city','owner_city_level','brand','user_level','sex','family_client','group_client','music','video_member','out_prdct_fee','out_prdct_fee_avg','point_fee']
#ClassifyColumnList=['netlong','card_type','owner_city','brand','user_level','sex','family_client','group_client','music','video_member','owner_city_level','out_prdct_fee','out_prdct_fee_avg','point_fee','other_tv','finance','bank']
#ClassifyColumnList=['netlong', 'out_prdct_fee', 'point_fee', 'brand', 'user_level', 'sex', 'family_client', 'group_client', 'owner_city_level']
ClassifyColumnList=['prdct_fee_derive_1', 'netlong_derive_1',
       'listing_price_derive_1', 'listing_data_months_derive_1',
       'battery_capacity_derive_1', 'arpu_derive_1',
       'brand', 'card_type', 'family_client', 'finance',
       'group_client', 'listing_data_months', 'music',
       'netlong', 'out_prdct_fee', 'out_prdct_fee_avg', 'owner_city',
       'owner_city_level', 'screen_size', 'sex', 'storage',
       'user_level', 'video_member']
writer = pd.ExcelWriter('0702广发信用卡模型用户分类变量反馈率统计.xlsx')

for ClassifyColumn in ClassifyColumnList:
    columnvaluecount_dict[ClassifyColumn]['ColumnStatistics'].to_excel(excel_writer=writer, sheet_name=ClassifyColumn, encoding="utf-8")
    #print(ClassifyColumn,columnvaluecount_dict[ClassifyColumn]['ColumnStatistics'])
writer.save()
writer.close()

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.font_manager import FontProperties
myfont=FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf',size=10)
sns.set(font=myfont.get_name())

# 对连续型变量的反馈率数据进行绘图操作：
ContinuousColumnList=['prdct_fee_derive_1', 'netlong_derive_1',
       'listing_price_derive_1', 'listing_data_months_derive_1',
       'battery_capacity_derive_1', 'arpu_derive_1',]
for ContinuousColumn in ContinuousColumnList:
    #print(columnvaluecount_dict[ContinuousColumn]['TurnSampleLogPercentRadio'])
    ContinuousSampleLogPercentRadio=pd.DataFrame(columnvaluecount_dict[ContinuousColumn]['TurnSampleLogPercentRadio'])
    #print(ContinuousSampleLogPercentRadio.sort_values(by=ContinuousColumn,ascending=False).iloc[1])
    ContinuousSampleLogPercentRadio.replace(float('inf'),ContinuousSampleLogPercentRadio.sort_values(by=ContinuousColumn,ascending=False).iloc[1],inplace=True)
    ContinuousSampleLogPercentRadio['X_value']=np.log(ContinuousSampleLogPercentRadio.index) #进行对数变换
    #ContinuousSampleLogPercentRadio['X_value']=ContinuousSampleLogPercentRadio.index
    #print(ContinuousSampleLogPercentRadio,ContinuousSampleLogPercentRadio.columns)
    g = sns.FacetGrid(ContinuousSampleLogPercentRadio,palette="Set1", size=7)
    g.map(plt.scatter,'X_value',ContinuousColumn,linewidth=.8)
    print(ContinuousColumn,ContinuousSampleLogPercentRadio)
    g.set_xticklabels(rotation=90)
    g.set_axis_labels(ContinuousColumn, "log(p/1-p)");
    #plt.xlim(1,8)
    plt.title(ContinuousColumn+u'(反馈率图形)')
    g.add_legend()
    g.savefig('广发信用卡模型连续型变量反馈率图像(高消费用户)\\'+ContinuousColumn+u'(反馈率图形).png',dpi=100)

In [ ]:
xy_1.columns

In [ ]:
plt.style.use('bmh')
plt.rc('font',family = 'SimHei',size = 13)
cat_list = ['target', 'card_type', 'netlong', 'owner_city', 'arpu',
       'out_prdct_fee', 'prdct_fee', 'point_fee', 'myth_fee', 'brand',
       'listing_date', 'listing_price', 'nettime', 'user_level', 'sex',
       'family_client', 'group_client', 'music', 'expirationtime',
       'video_member', 'other_tv', 'finance', 'bank', 'campus_client',
       'migu_tv', 'age', 'adis', 'arpu_avg', 'myth_fee_avg',
       'out_prdct_fee_avg', 'prdct_fee_avg', 'os', 'battery_capacity',
       'storage', 'screen_size']
for n,i in enumerate(cat_list):
    xy_1_cat_num = xy_1[i].value_counts().index.shape[0]
    print('{0}.{1}特征的类型数量是：{2}'.format(n+1,i,xy_1_cat_num))

In [ ]:
ChuanQi_basic_df.columns

In [ ]:
# 删除 listing_data_months_derive_1，owner_city，listing_data_months

xy_1=ChuanQi_basic_df[['target', 'arpu',
       'battery_capacity', 'brand', 'card_type', 'family_client', 'finance',
       'group_client', 'listing_price', 'music',
       'netlong', 'out_prdct_fee', 'out_prdct_fee_avg',
       'owner_city_level', 'prdct_fee', 'screen_size', 'sex', 'storage',
       'user_level', 'video_member']]

In [ ]:
msno.matrix(xy_1,labels = True)

In [ ]:
pandas_profiling.ProfileReport(xy_1)

In [ ]:
for columns in ['target', 'prdct_fee_derive_1', 'netlong_derive_1',
       'listing_price_derive_1',
       'battery_capacity_derive_1', 'arpu_derive_1', 'arpu',
       'battery_capacity', 'brand', 'card_type', 'family_client', 'finance',
       'group_client', 'listing_price', 'music',
       'netlong', 'out_prdct_fee', 'out_prdct_fee_avg',
       'owner_city_level', 'prdct_fee', 'screen_size', 'sex', 'storage',
       'user_level', 'video_member']:
    print(columns,len(xy_1[columns].value_counts()))

In [ ]:
#去除掉以上列
xy_6 = xy_1[['target','arpu','brand','family_client','finance','group_client','listing_price','music','netlong','out_prdct_fee_avg','owner_city_level','prdct_fee','screen_size','sex','storage','user_level','video_member']]

# 连续型特征变量：['netlong','arpu','prdct_fee','listing_price','arpu_avg','prdct_fee_avg','screen_size']
# 去除重复行：
xy_6.drop_duplicates(keep='first',inplace=True)
xy_6.reset_index(inplace=True)

In [ ]:

xy_6['netlong'][(xy_6['netlong']==9999)]=58
xy_6['netlong'][(xy_6['netlong']==0)]=31

xy_6['arpu'][(xy_6['arpu']>400)&(xy_6['arpu']<9999)]=400
xy_6['arpu'][(xy_6['arpu']==9999)]=500

xy_6['prdct_fee'][(xy_6['prdct_fee']==0)]=70
xy_6['prdct_fee'][(xy_6['prdct_fee']>500)&(xy_6['prdct_fee']<9999)]=500
xy_6['prdct_fee'][(xy_6['prdct_fee']==9999)]=500

xy_6['listing_price'][(xy_6['listing_price']==9999)]=500

# xy_6['arpu_avg'][(xy_6['arpu_avg']>500)&(xy_6['arpu_avg']<9999)]=500
# xy_6['arpu_avg'][(xy_6['arpu_avg']==9999)]=400

# xy_6['prdct_fee_avg'][(xy_6['prdct_fee_avg']>500)&(xy_6['prdct_fee_avg']<9999)]=500
# xy_6['prdct_fee_avg'][(xy_6['prdct_fee_avg']==9999)]=400

xy_6['screen_size'][(xy_6['screen_size']==99)]=0.37
xy_6['screen_size'][~(xy_6['screen_size'].isin([6,5.9,5.7,4.7,5.5,5.2,4.5,4,5,99,2.4,]))]=2

# # 电池容量：battery_capacity_derive_1
# batterymap={3025:1,5845:1,2875:1,2550:2,3260:2,1850:2,2175:3,7137.5:3,850.4995:3}
# xy_6['battery_capacity_derive_1'].replace(batterymap,inplace=True)

# 手机品牌：
brandmap={1:1,2:2,5:2,6:2,7:3,8:3,4:3,9:3,3:4,10:4,11:4,12:4}
xy_6['brand'].replace(brandmap,inplace=True)
# xy_6['brand'][(xy_6['brand']==8)|(xy_6['brand']==10)|(xy_6['brand']==6)] = 1000
# xy_6['brand'][(xy_6['brand']==11)|(xy_6['brand']==7)|(xy_6['brand']==12)|(xy_6['brand']==4)] = 2000
# xy_6['brand'][(xy_6['brand']==9)|(xy_6['brand']==3)|(xy_6['brand']==1)|(xy_6['brand']==2)|(xy_6['brand']==5)] = 3000

# user_level
userlevelmap={4:1,5:1,6:1,7:1,3:2,9999:3,0:4,1:4,2:4}
xy_6['user_level'].replace(userlevelmap,inplace=True)
# xy_6['user_level'][(xy_6['user_level']>=5) & (xy_6['user_level']<=7)] =100
# xy_6['user_level'][(xy_6['user_level']<=4) | (xy_6['user_level']==9999)] = 200

# 内存表：

storagemap={'64GB':1,'128GB':1,'32GB':1,'16GB':1,'4GB':2,'8GB':2,'512MB':2,'nan':2,'256MB':3,'128MB':3,'4MB':3,'其它存储':3,'32MB':3,'64MB':3,'16MB':3,'8MB':3}
xy_6['storage'].replace(storagemap,inplace=True)

# owner_city_level
citylevelmap={8:1,9:1,9999:1,7:2,6:2,10:2,1:3,5:3,4:3,3:4,2:4}
xy_6['owner_city_level'].replace(citylevelmap,inplace=True)
# xy_6['owner_city_level'][xy_6['owner_city_level']>=7] =3
# xy_6['owner_city_level'][(xy_6['owner_city_level']>=4) & (xy_6['owner_city_level']<7)] = 2
# xy_6['owner_city_level'][xy_6['owner_city_level']<=3] = 1



In [ ]:
xy_6.columns

In [ ]:
for columns in [ 'target', 'arpu', 'brand', 'family_client',
       'finance', 'group_client', 'listing_price', 'music', 'netlong',
       'out_prdct_fee_avg', 'owner_city_level', 'prdct_fee', 'screen_size',
       'sex', 'storage', 'user_level', 'video_member']:
    print(columns,len(xy_6[columns].value_counts()))

In [ ]:
# 对模型的离散型变量进行虚拟变量处理：onehot编码需要str格式
ClassifyColumnsList=['brand', 'family_client',
       'finance', 'group_client', 'music', 'out_prdct_fee_avg',
       'owner_city_level', 'sex', 'storage', 'user_level',
       'video_member']

for ClassifyColumn in ClassifyColumnsList:
    #print(ClassifyColumn,len(ChuanQi_basic_df[ClassifyColumn].value_counts()))
    xy_6[ClassifyColumn]=xy_6[ClassifyColumn].astype('str')
xy_7 = xy_6[['target', 'arpu', 'brand', 'family_client', 'finance',
       'group_client', 'listing_price', 'music', 'netlong',
       'out_prdct_fee_avg', 'owner_city_level', 'prdct_fee', 'screen_size',
       'sex', 'storage', 'user_level', 'video_member']]
##筛选特定列
display(xy_7.info())

In [ ]:
xy_7_onehot=pd.get_dummies(xy_7)
xy_7_onehot.columns

In [ ]:
np.isnan(xy_7_onehot).any()#False表示无缺失值
xy_7_onehot[xy_7_onehot.isnull().values==True]

In [ ]:
from sklearn import linear_model as lr
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import sklearn.model_selection as ms
import sklearn.metrics as sm
class_weight = dict({1:9,0:1})#配置权重，1万正样本权重为9,9万随机样本的权重为1
df_x = xy_7_onehot[[ 'arpu', 'listing_price', 'netlong', 'prdct_fee',
       'screen_size', 'brand_1', 'brand_2', 'brand_3', 'brand_4',
       'family_client_1.0', 'family_client_9999.0', 'finance_1.0',
       'finance_9999.0', 'group_client_1.0', 'group_client_9999.0',
       'music_1.0', 'music_9999.0', 'out_prdct_fee_avg_0.0',
       'out_prdct_fee_avg_1.0', 'out_prdct_fee_avg_9999.0',
       'owner_city_level_1.0', 'owner_city_level_2.0', 'owner_city_level_3.0',
       'owner_city_level_4.0', 'sex_0.0', 'sex_1.0', 'sex_9999.0', 'storage_1',
       'storage_2', 'storage_3', 'user_level_1.0', 'user_level_2.0',
       'user_level_3.0', 'user_level_4.0', 'video_member_1.0',
       'video_member_9999.0']]
df_y = xy_7_onehot['target']
X_train, X_test, Y_train, Y_test = train_test_split(df_x, df_y, test_size=0.5, random_state=0)

#逻辑回归
model = linear_model.LogisticRegression(class_weight = class_weight)
# model = linear_model.Lasso()
model.fit(X_train, Y_train)
y_train_score = model.decision_function(X_train)#用于计算训练集的KS值
y_score = model.decision_function(X_test)#计算预测结果概率值，用于计算测试集的KS值
y_test = model.predict(X_test)
print('模型预测成功率为：',(y_test == Y_test).sum() / len(y_test))
y_test1 = pd.Series(y_test)#**numpy没有value_counts,故要转化为pd.Series.此处是预测值
Y_test1 = pd.Series(Y_test)
print('真实测试数据：\n',Y_test1.value_counts())
print('预测的测试数据：\n', y_test1.value_counts())

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegressionCV,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model.coordinate_descent import ConvergenceWarning
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

#对数据的训练集进行标准化
ss = StandardScaler()
X_StandardScaler = ss.fit(X_train)
X_train=X_StandardScaler.transform(X_train)
X_test= X_StandardScaler.transform(X_test)
# X_train = ss.fit_transform(X_train)     #先拟合数据在进行标准化
# X_test = ss.fit_transform(X_test)
display(len(X_train),len(X_test))


lr = LogisticRegressionCV(multi_class="ovr",fit_intercept=True,Cs=np.logspace(-2,2,20),cv=2,penalty="l2",solver="lbfgs",tol=0.01)

re = lr.fit(X_train,Y_train)
r_train = re.score(X_train,Y_train)


print("R值(准确率):",r_train)
print("参数:",re.coef_)
print("截距:",re.intercept_)
print("稀疏化特征比率:%.2f%%" %(np.mean(lr.coef_.ravel()==0)*100))
print("=========sigmoid函数转化的值，即：概率p=========")
print(re.predict_proba(X_test)[:5])     #sigmoid函数转化的值，即：概率p


Y_test_predict = lr.predict(X_test)      #预测

# 计算测试集的准确率···
r_test = re.score(X_test,Y_test)
print("R值(准确率):",r_test)
display(Y_test_predict.sum())

In [ ]:
# 绘制模型的Roc/Auc值的曲线图

from sklearn import metrics

lr_y_test_score = lr.decision_function(X_test)
lr_fpr_test,lr_tpr_test,lr_threasholds_test = metrics.roc_curve(Y_test.ravel(),lr_y_test_score.ravel())
lr_auc_test = metrics.auc(lr_fpr_test,lr_tpr_test)

lr_y_train_score = lr.decision_function(X_train)
lr_fpr_train,lr_tpr_train,lr_threasholds_train = metrics.roc_curve(Y_train.ravel(),lr_y_train_score.ravel())  #Y_predict
lr_auc_train = metrics.auc(lr_fpr_train,lr_tpr_train)


print("Logistic算法R值(训练):",lr.score(X_train,Y_train))
print("Logistic算法AUC值(训练):",lr_auc_train)

print("Logistic算法R值(测试):",lr.score(X_test,Y_test))
print("Logistic算法AUC值(测试):",lr_auc_test)

In [ ]:
# 在训练集上统计十等分检验表：
Y_train_proba=lr.predict_proba(X_train)
Y_train_predict=lr.predict(X_train)
Y_train_predict_true=Y_train_proba[:,1]

Y_test_proba=lr.predict_proba(X_test)
Y_test_predict=lr.predict(X_test)
Y_test_predict_true=Y_test_proba[:,1]
Y_train_predict.sum()

In [ ]:
# 训练集的十等分表：
IndexSeriers=pd.Series(np.arange(0,10,1))


Result_KS_10_Dic={'Train_KS_10':{},'Test_KS_10':{}}
Result_KS_10_Dic['Train_KS_10']['y_train']=Y_train
Result_KS_10_Dic['Train_KS_10']['y_train_predict']=Y_train_predict
Result_KS_10_Dic['Train_KS_10']['y_train_true_proba']=Y_train_predict_true
Train_KS_10_df=pd.DataFrame(Result_KS_10_Dic['Train_KS_10'])

Result_KS_10_Dic['Test_KS_10']['y_test']=Y_test
Result_KS_10_Dic['Test_KS_10']['y_test_predict']=Y_test_predict
Result_KS_10_Dic['Test_KS_10']['y_test_true_proba']=Y_test_predict_true
Test_KS_10_df=pd.DataFrame(Result_KS_10_Dic['Test_KS_10'])
display(Train_KS_10_df.head(),len(Train_KS_10_df),Test_KS_10_df.head(),len(Test_KS_10_df))
# 对概率值进行降序排列：
Train_KS_10_df.sort_values('y_train_true_proba',ascending=False,inplace=True)
Train_KS_10_df.reset_index(inplace=True)
Train_KS_10_df['true_proba_cluster']=pd.qcut(Train_KS_10_df.y_train_true_proba,10)
Result_KS_10_Dic['KS_10_cluster_crowd_number']=pd.DataFrame(Train_KS_10_df.true_proba_cluster.value_counts().sort_index(ascending=False))
Result_KS_10_Dic['KS_10_cluster_crowd_number'].reset_index(inplace=True)
Result_KS_10_Dic['KS_10_cluster_crowd_number']=pd.concat([Result_KS_10_Dic['KS_10_cluster_crowd_number'],IndexSeriers],axis=1)
display(Result_KS_10_Dic['KS_10_cluster_crowd_number'])

# 对测试值进行降序排列：
Test_KS_10_df.sort_values('y_test_true_proba',ascending=False,inplace=True)
Test_KS_10_df.reset_index(inplace=True)
Test_KS_10_df['true_proba_cluster']=pd.qcut(Test_KS_10_df.y_test_true_proba,10)
Result_KS_10_Dic['Test_KS_10_cluster_crowd_number']=pd.DataFrame(Test_KS_10_df.true_proba_cluster.value_counts().sort_index(ascending=False))
Result_KS_10_Dic['Test_KS_10_cluster_crowd_number'].reset_index(inplace=True)
Result_KS_10_Dic['Test_KS_10_cluster_crowd_number']=pd.concat([Result_KS_10_Dic['Test_KS_10_cluster_crowd_number'],IndexSeriers],axis=1)
display(Result_KS_10_Dic['Test_KS_10_cluster_crowd_number'])


In [ ]:
# 计算KS_10对应的分层映射值：
Result_KS_10_Dic['KS_10_cluster_map']={}
for index_number in range(len(Result_KS_10_Dic['KS_10_cluster_crowd_number'])):
    Result_KS_10_Dic['KS_10_cluster_map'][Result_KS_10_Dic['KS_10_cluster_crowd_number']['index'][index_number]]=index_number
display(Result_KS_10_Dic['KS_10_cluster_map'])

# 计算KS_10对应的分层映射值：
Result_KS_10_Dic['Test_KS_10_cluster_map']={}
for index_number in range(len(Result_KS_10_Dic['Test_KS_10_cluster_crowd_number'])):
    Result_KS_10_Dic['Test_KS_10_cluster_map'][Result_KS_10_Dic['Test_KS_10_cluster_crowd_number']['index'][index_number]]=index_number
display(Result_KS_10_Dic['Test_KS_10_cluster_map'])

In [ ]:
Train_KS_10_df['true_proba_cluster_map']=Train_KS_10_df['true_proba_cluster'].replace(Result_KS_10_Dic['KS_10_cluster_map'])

Test_KS_10_df['true_proba_cluster_map']=Test_KS_10_df['true_proba_cluster'].replace(Result_KS_10_Dic['Test_KS_10_cluster_map'])

Train_KS_10_df_groupby_true_proba_cluster_map=Train_KS_10_df.groupby('true_proba_cluster_map')

Test_KS_10_df_groupby_true_proba_cluster_map=Test_KS_10_df.groupby('true_proba_cluster_map')

Result_KS_10_Dic['KS_10_TPV_Number']={}
for proba_cluster_map,proba_cluster_value_df in Train_KS_10_df_groupby_true_proba_cluster_map:
    Result_KS_10_Dic['KS_10_TPV_Number'][proba_cluster_map]=(Train_KS_10_df_groupby_true_proba_cluster_map.get_group(proba_cluster_map)['y_train']==1).sum()
    
Result_KS_10_Dic['Test_KS_10_TPV_Number']={}
for proba_cluster_map,proba_cluster_value_df in Test_KS_10_df_groupby_true_proba_cluster_map:
    Result_KS_10_Dic['Test_KS_10_TPV_Number'][proba_cluster_map]=(Test_KS_10_df_groupby_true_proba_cluster_map.get_group(proba_cluster_map)['y_test']==1).sum()


Result_KS_10_Dic['Real_True_Sample_number']=pd.Series(Result_KS_10_Dic['KS_10_TPV_Number'])
Result_KS_10_Dic['Real_True_Sample_number_rate3']=Result_KS_10_Dic['Real_True_Sample_number']/Result_KS_10_Dic['KS_10_cluster_crowd_number']['true_proba_cluster']
Result_KS_10_Dic['Real_True_Sample_number_rate4']=pd.Series(Result_KS_10_Dic['Real_True_Sample_number']/np.array(list(Result_KS_10_Dic['KS_10_TPV_Number'].values())).sum())

#Result_KS_10_Dic['Test_KS_10_TPV_Number']
Result_KS_10_Dic['Test_Real_True_Sample_number']=pd.Series(Result_KS_10_Dic['Test_KS_10_TPV_Number'])
Result_KS_10_Dic['Test_Real_True_Sample_number_rate3']=Result_KS_10_Dic['Test_Real_True_Sample_number']/Result_KS_10_Dic['Test_KS_10_cluster_crowd_number']['true_proba_cluster']
Result_KS_10_Dic['Test_Real_True_Sample_number_rate4']=pd.Series(Result_KS_10_Dic['Test_Real_True_Sample_number']/np.array(list(Result_KS_10_Dic['Test_KS_10_TPV_Number'].values())).sum())


Result_KS_10_Statistics_df=pd.concat([Result_KS_10_Dic['KS_10_cluster_crowd_number']['true_proba_cluster'],Result_KS_10_Dic['Real_True_Sample_number'],Result_KS_10_Dic['Real_True_Sample_number_rate3'],Result_KS_10_Dic['Real_True_Sample_number_rate4']],axis=1)
Result_KS_10_Statistics_df.columns=['CustomNumber','True_Sample_CustomNumber','Real_True_Sample_number_rate3','Real_True_Sample_number_rate4']
Result_KS_10_Statistics_df['Real_True_Sample_number_rate4_accumulate']=Result_KS_10_Statistics_df['Real_True_Sample_number_rate4'].cumsum()
Result_KS_10_Statistics_df['Real_True_Sample_number_rate_random']=np.linspace(0,Result_KS_10_Statistics_df['Real_True_Sample_number_rate4_accumulate'].max(),10)
Result_KS_10_Statistics_df['ModelAcquireVaue']=Result_KS_10_Statistics_df['Real_True_Sample_number_rate4_accumulate']-Result_KS_10_Statistics_df['Real_True_Sample_number_rate_random']


Result_KS_10_Statistics_Test_df=pd.concat([Result_KS_10_Dic['Test_KS_10_cluster_crowd_number']['true_proba_cluster'],Result_KS_10_Dic['Test_Real_True_Sample_number'],Result_KS_10_Dic['Test_Real_True_Sample_number_rate3'],Result_KS_10_Dic['Test_Real_True_Sample_number_rate4']],axis=1)
Result_KS_10_Statistics_Test_df.columns=['Test_CustomNumber','Test_True_Sample_CustomNumber','Test_Real_True_Sample_number_rate3','Test_Real_True_Sample_number_rate4']
Result_KS_10_Statistics_Test_df['Test_Real_True_Sample_number_rate4_accumulate']=Result_KS_10_Statistics_Test_df['Test_Real_True_Sample_number_rate4'].cumsum()
Result_KS_10_Statistics_Test_df['Test_Real_True_Sample_number_rate_random']=np.linspace(0,Result_KS_10_Statistics_Test_df['Test_Real_True_Sample_number_rate4_accumulate'].max(),10)
Result_KS_10_Statistics_Test_df['Test_ModelAcquireVaue']=Result_KS_10_Statistics_Test_df['Test_Real_True_Sample_number_rate4_accumulate']-Result_KS_10_Statistics_Test_df['Test_Real_True_Sample_number_rate_random']



In [ ]:
Result_KS_10_Dic['KS_Point']={'x_point':Result_KS_10_Statistics_df['ModelAcquireVaue'].idxmax(),'y_point':Result_KS_10_Statistics_df['Real_True_Sample_number_rate4_accumulate'][Result_KS_10_Statistics_df['ModelAcquireVaue'].idxmax()]}
display(Result_KS_10_Dic['KS_Point'])

Result_KS_10_Dic['Test_KS_Point']={'x_point':Result_KS_10_Statistics_Test_df['Test_ModelAcquireVaue'].idxmax(),'y_point':Result_KS_10_Statistics_Test_df['Test_Real_True_Sample_number_rate4_accumulate'][Result_KS_10_Statistics_Test_df['Test_ModelAcquireVaue'].idxmax()]}
display(Result_KS_10_Dic['Test_KS_Point'])

In [ ]:
#每个箱子中箱子正样本比例Real_True_Sample_number_rate3=True_Sample_CustomNumber/ CustomNumber
#箱子正样本占总正样本比例Real_True_Sample_number_rate4=True_Sample_CustomNumber/ΣTrue_Sample_CustomNumber
#累计和Real_True_Sample_number_rate4_accumulate=ΣReal_True_Sample_number_rate4
display(Result_KS_10_Statistics_df,Result_KS_10_Statistics_Test_df)

In [ ]:
#显著性分析代码：p>|z|大于0.01的进入模型
#AUC（模型稳定性）和KS（区分能力）值计算
# Coef. :参数、系数
# Std. Err.：系数Coef.的方差的平方根standard error
# [95% Conf. Interval] 系数Coef.的95%置信区间
# P>|z|: 值小于或等于0.05表示结果显著。 
#两个变量协方差分析相关性强，保留Z值绝对值大（显著性强）的那个，如brand和list_price相关性强，就把Z值大的brand2留下
import os
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import auc,roc_curve
import matplotlib.pyplot as plt
X_train, X_test, Y_train, Y_test = train_test_split(df_x, df_y, test_size=0.5, random_state=0)
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out = 0.05,
                       verbose = True):
    included = list(initial_list)
 
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
 
        # backward step
        model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included
 
result = stepwise_selection(X_train, Y_train)#模板改这里的参数X_train, Y_train
 
print('resulting features:')
print(result)

lr = sm.Logit(Y_train,sm.add_constant(X_train[result]))
rst = lr.fit()
print(rst.summary2())

y_predicted = rst.predict(sm.add_constant(X_train[result]))

fpr, tpr, thresholds = roc_curve(Y_train,y_predicted, pos_label=1)
auc_score = auc(fpr,tpr)
w = tpr - fpr
ks_score = w.max()
ks_x = fpr[w.argmax()]
ks_y = tpr[w.argmax()]
fig,ax = plt.subplots()
ax.plot(fpr,tpr,label='AUC=%.5f'%auc_score)
ax.set_title('Receiver Operating Characteristic')
ax.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
ax.plot([ks_x,ks_x], [ks_x,ks_y], '--', color='red')
ax.text(ks_x,(ks_x+ks_y)/2,'  KS=%.5f'%ks_score)
ax.legend()
fig.show()  